<a href="https://colab.research.google.com/github/Dzeno0405/Smart-Email-Classifier-Responder/blob/main/Smart_Email_Classifier_and_Responder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project takes emails, figures out what they’re about (support, sales, or feedback), and then writes a short, professional reply automatically.
In Colab, I set up a FastAPI server to run the code, loaded AI models to classify and respond to emails, and used ngrok to create a temporary public URL so the React webpage can talk to the server.
When someone enters an email on the webpage, it sends it to our Colab server, the AI processes it, and sends back a category and a reply.
I added a health check to see if the server is alive, CORS so any webpage can connect, and some filtering so the AI replies stay clean and professional.
Basically, it’s a live demo of how AI can handle customer emails end-to-end.

In [7]:
!pip install openai pandas transformers

In [11]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-..."
client = OpenAI()


In [19]:
emails = [
    # --- Support (17) ---
    "My package arrived damaged, I need a replacement.",
    "I forgot my password, how can I reset it?",
    "My payment failed, can you help me process it?",
    "The app keeps crashing when I try to log in.",
    "I never received my confirmation email.",
    "I want to change the shipping address for my order.",
    "The item I received is not what I ordered.",
    "I’m having trouble logging into my account.",
    "I was charged twice for my last purchase.",
    "How do I cancel my subscription immediately?",
    "The website shows an error when I try to checkout.",
    "I didn’t receive the tracking number for my order.",
    "I’m unable to update my account details.",
    "My invoice has incorrect information.",
    "I need help recovering deleted data from my account.",
    "The product I received is defective.",
    "I’m experiencing connectivity issues with your app.",

    # --- Sales (17) ---
    "Do you offer discounts for bulk orders?",
    "I want to know about your premium subscription plans.",
    "Can we schedule a call to discuss partnership opportunities?",
    "What are your prices for enterprise clients?",
    "Do you provide a free trial of your service?",
    "Are there any upcoming promotions for new customers?",
    "Can I get a quote for a large-scale deployment?",
    "Do you offer student or educational discounts?",
    "I’m interested in your reseller program.",
    "Can you provide details about your referral program?",
    "Do you have volume-based pricing for your software?",
    "Is there a loyalty program for repeat customers?",
    "Can I customize the package to include extra features?",
    "Do you provide training or onboarding for new clients?",
    "Can I schedule a demo of your platform?",
    "Are there seasonal deals available?",
    "What payment plans do you offer for long-term subscriptions?",

    # --- Feedback (17) ---
    "Great service! I'm really happy with your support team.",
    "Your app is good but needs a dark mode option.",
    "The checkout process is confusing and needs improvement.",
    "I love the new update, keep up the good work!",
    "Your delivery times have improved significantly, thanks!",
    "I appreciate the detailed product documentation.",
    "The interface is intuitive and easy to use.",
    "Customer service was very helpful and polite.",
    "I experienced some bugs but overall the app is great.",
    "Thank you for promptly resolving my last issue.",
    "I like the new features added this month.",
    "Your tutorials were very clear and helpful.",
    "The mobile app works smoothly and is responsive.",
    "I would suggest adding more payment options.",
    "Your newsletter is informative and well-written.",
    "I had an issue but the resolution was fast and effective.",
    "Keep up the consistent updates, they’re very helpful."
]



In [20]:
from transformers import pipeline

# Create a zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define categories
labels = ["Support", "Sales", "Feedback"]

results = []
for email in emails:
    result = classifier(email, candidate_labels=labels)
    category = result["labels"][0]  # Best-matching label
    print(f"Email: {email}\nCategory: {category}\n{'-'*40}")
    results.append({"email": email, "category": category})


Device set to use cpu


Email: My package arrived damaged, I need a replacement.
Category: Support
----------------------------------------
Email: I forgot my password, how can I reset it?
Category: Support
----------------------------------------
Email: My payment failed, can you help me process it?
Category: Support
----------------------------------------
Email: The app keeps crashing when I try to log in.
Category: Feedback
----------------------------------------
Email: I never received my confirmation email.
Category: Support
----------------------------------------
Email: I want to change the shipping address for my order.
Category: Feedback
----------------------------------------
Email: The item I received is not what I ordered.
Category: Feedback
----------------------------------------
Email: I’m having trouble logging into my account.
Category: Feedback
----------------------------------------
Email: I was charged twice for my last purchase.
Category: Sales
----------------------------------------

In [21]:
from transformers import pipeline

# Load a text-to-text generation model (good for instructions)
generator = pipeline("text2text-generation", model="google/flan-t5-base")

def generate_response(email_text, category):
    prompt = f"Write a polite {category.lower()} response to this email:\n\n{email_text}"
    result = generator(prompt, max_length=100, do_sample=False)
    return result[0]["generated_text"]

for r in results:
    r["auto_response"] = generate_response(r["email"], r["category"])

# Show results
for r in results:
    print(f"Email: {r['email']}")
    print(f"Category: {r['category']}")
    print(f"Auto Response: {r['auto_response']}")
    print("-"*50)


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

Email: My package arrived damaged, I need a replacement.
Category: Support
Auto Response: I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that.
--------------------------------------------------
Email: I forgot my password, how can I reset it?
Category: Support
Auto Response: How can I

In [22]:
import time

def classify_and_measure(email):
    start = time.time()
    result = classifier(email, candidate_labels=labels)
    end = time.time()
    return result["labels"][0], round(end - start, 3)  # seconds as "cost"

results = []
for email in emails:
    category, cost = classify_and_measure(email)
    auto_response = generate_response(email, category)
    results.append({"email": email, "category": category, "auto_response": auto_response, "cost_sec": cost})

pd.DataFrame(results)


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


,email,category,auto_response,cost_sec
0,"My package arrived damaged, I need a replacement.",Support,I'm sorry to hear that. I'm sorry to hear that...,2.050
1,"I forgot my password, how can I reset it?",Support,How can I reset my password?,2.181
2,"My payment failed, can you help me process it?",Support,I'm sorry to hear that. I'm sorry to hear that...,1.419
3,The app keeps crashing when I try to log in.,Feedback,Is there anything I can do to fix this? Is the...,1.485
4,I never received my confirmation email.,Support,I'm sorry to hear that. I'm sorry to hear that...,1.259
5,I want to change the shipping address for my o...,Feedback,I'm sorry to hear that. I'm sorry to hear that...,1.430
6,The item I received is not what I ordered.,Feedback,I'm sorry to hear that. I'm sorry to hear that...,1.392
7,I’m having trouble logging into my account.,Feedback,Is there anything else I can help you with? Is...,1.480
8,I was charged twice for my last purchase.,Sales,I was charged twice for my last purchase.,1.313
9,How do I cancel my subscription immediately?,Feedback,You can cancel your subscription at any time b...,1.296


In [1]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers


In [2]:
!ngrok authtoken 31nFq2Ds6yJgLekfn6aALykDIQI_4mHMqymQupJjT3mkEYeo7


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install pyngrok --quiet
from pyngrok import ngrok


In [4]:
public_url = ngrok.connect(8001)  # new port
print(f"Public URL: {public_url.public_url}")


Public URL: https://5b53c08350ce.ngrok-free.app


In [5]:
from transformers import pipeline

# Load text generation pipeline
generator = pipeline("text-generation", model="gpt2")

def generate_response(email_text):
    prompt = f"The user wrote: {email_text}\nWrite a short, polite reply:"
    result = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)
    return result[0]['generated_text'].split("Write a short, polite reply:")[-1].strip()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [6]:
# STEP 1: Imports
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
import nest_asyncio
from pyngrok import ngrok
import threading
import re
from fastapi.middleware.cors import CORSMiddleware


# STEP 2: Colab / Jupyter setup
nest_asyncio.apply()
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],          # for demos; restrict later
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/healthz")
def health():
    return {"ok": True}

# STEP 3: Load models
print("Loading models... this may take a few seconds...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
generator = pipeline("text2text-generation", model="google/flan-t5-large")
labels = [
    "Customer support request",
    "Sales inquiry about pricing or plans",
    "Feedback or suggestion"
]
print("Models loaded!")

# STEP 4: API request model
class EmailRequest(BaseModel):
    email: str

# STEP 5: Echo checker
def is_echoy(response: str, email: str) -> bool:
    r = re.sub(r'\s+', ' ', response.lower()).strip()
    e = re.sub(r'\s+', ' ', email.lower()).strip()
    if e in r or r in e:
        return True
    e_words = set(e.split())
    if not e_words:
        return False
    overlap = len(e_words & set(r.split())) / len(e_words)
    return overlap > 0.6

# STEP 6: Category instructions
category_instructions = {
    "Support": "Ask for order number, screenshots, account email if needed. Provide short troubleshooting and explain next steps.",
    "Sales": "Ask if request is for individual/team/enterprise. Offer pricing sheet or demo info. Do NOT invent prices.",
    "Feedback": "Thank the user, summarize feedback briefly, mention forwarding to product team or follow-up."
}

# STEP 7: Few-shot examples
few_shot_examples = [
    {
        "email": "My package arrived damaged and I need a replacement.",
        "reply": "Sorry to hear that. Please provide your order number and a photo of the damage so we can arrange a replacement."
    },
    {
        "email": "Do you offer discounts for teams or bulk purchases?",
        "reply": "Thanks for asking! Could you share your team size so we can provide accurate pricing information?"
    },
    {
        "email": "I really like your new app update, but the interface is confusing.",
        "reply": "Thank you for your feedback. We'll share this with our product team for review."
    }
]

# STEP 8: Classify & respond endpoint
@app.post("/classify")
def classify_email(request: EmailRequest):
    # Classify email
    classification_result = classifier(request.email, candidate_labels=labels)
    label_full = classification_result["labels"][0]

    # Map to simple category
    if "support" in label_full.lower():
        category_simple = "Support"
    elif "sales" in label_full.lower():
        category_simple = "Sales"
    else:
        category_simple = "Feedback"

    instructions = category_instructions.get(category_simple, "")

    # Build prompt with strict agent role
    prompt = (
        "You are a professional customer support agent.\n"
        "You must only reply as support. Do NOT act as the customer. Do NOT repeat the customer's words.\n"
        "Write a short, polite, professional, and helpful response.\n\n"
    )

    for ex in few_shot_examples:
        prompt += f"Customer email: {ex['email']}\nReply: {ex['reply']}\n\n"

    prompt += f"Customer email: {request.email}\nReply:"

    # Generate response
    response_result = generator(
        prompt,
        max_new_tokens=80,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        no_repeat_ngram_size=3
    )

    auto_response = response_result[0]["generated_text"].strip()
    if "Reply:" in auto_response:
        auto_response = auto_response.split("Reply:")[-1].strip()

    # Deduplicate repeated phrases
    lines = auto_response.split(". ")
    seen = set()
    cleaned_lines = []
    for line in lines:
        if line and line not in seen:
            cleaned_lines.append(line)
            seen.add(line)
    auto_response = ". ".join(cleaned_lines).strip()

    # Fallback for echo or too short
    if is_echoy(auto_response, request.email) or len(auto_response) < 5:
        auto_response = "Thank you for reaching out. We'll get back to you with more details shortly."

    return {"email": request.email, "category": category_simple, "auto_response": auto_response}

# STEP 9: Start ngrok tunnel
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"  # replace with your token
!ngrok authtoken {NGROK_AUTH_TOKEN}
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url.public_url}")

# STEP 10: Run FastAPI in a separate thread
def run_app():
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_app).start()


Loading models... this may take a few seconds...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Models loaded!
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://c668156bbc03.ngrok-free.app


In [7]:
from pyngrok import ngrok
import requests

# Start tunnel
public_tunnel = ngrok.connect(8000)
public_url = public_tunnel.public_url  # ✅ get the string

print("Ngrok URL:", public_url)

# Now POST works correctly
email_test = {"email": "Hi, I want to know about your premium subscription plans."}
response = requests.post(f"{public_url}/classify", json=email_test)
print(response.json())


Ngrok URL: https://10ae7c2786d5.ngrok-free.app
INFO:     35.225.173.143:0 - "POST /classify HTTP/1.1" 200 OK
{'email': 'Hi, I want to know about your premium subscription plans.', 'category': 'Feedback', 'auto_response': 'I can help you with that. What would you like to know?'}


In [8]:
import requests

email_test = {"email": "Hi, I want to know about your premium subscription plans."}
response = requests.post(f"{public_url}/classify", json=email_test)
print(response.json())


INFO:     35.225.173.143:0 - "POST /classify HTTP/1.1" 200 OK
{'email': 'Hi, I want to know about your premium subscription plans.', 'category': 'Feedback', 'auto_response': 'I can help you with that. What do you want to know?'}
